In [1]:
import pandas as pd
import numpy as np
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import plotly.graph_objects as go

# Load data
df = pd.read_csv('sales_data_sample.csv' , encoding='unicode_escape')


# Initialize the app
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True

In [2]:
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [3]:
pd.set_option('display.max_columns', 50)

In [4]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,Motorcycles,95,S10_1678,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Questions To Be Ansewred ?!!
- What are the total sales for each year?
- what is the product with heighest sales in each country?
- what is the product with heighest sales in each country for every month?
- what is the country that has the heighest salse?

In [5]:
sales=list(df.groupby('MONTH_ID')['SALES'].mean().values)
months=list(range(1,13))
countries=list(df['COUNTRY'].unique())

In [6]:
colors = {
    'background': '#37003c',
    'sec_background':'#02884e',
    'text': '#fff'
}

In [7]:
app.layout = html.Div(
    children=[
     html.Br(),
     html.H1(children='SALES Exploration DASHBOARD',style={'font_family': 'Helvetica Neue','color': colors['text']
                   ,'textAlign':'center'}) ,  
     html.Div(
                 children=[
                    
                     
                      html.Div(className='six columns div-user-controls',
                      children=[  
                           html.Pre(children="Select Country", style={"fontSize":"150%"}),
                                dcc.Dropdown(
                                    id='country1',
                                    options=[
                                        {'label': i, 'value': i} for i in countries
                                    ],
                                    value='USA',
                                    style = {'width':'50%', 'height':'10%'}
                                ),
                                 
                                dcc.Graph(
                                        id='graph2',
                                        figure={}),
                                html.Br(),

                               
                           dcc.Graph(
                                        id='graph3',
                                        figure={})
                                            
                            
                                ]),html.Div(className='six columns div-user-controls',
    
                     children=[
                               
                                
                                        
                          html.Pre(children="Select Year", style={"fontSize":"150%"}),
                          dcc.Slider(
                                                    id='year',
                                                    min=2003,
                                                    max=2005,
                                                    marks={str(i): '{}'.format(str(i)) for i in [2003,2004,2005]},
                                                    value=2003
                                              
                                                    ),
                         dcc.Graph(
                                            id='graph1',
                                            figure={}
                                            ),html.Br(),
                               html.Br(),
                               dcc.Graph(
                                                id='Table',
                                   
                                                figure=go.Figure(
                                                    data=[go.Table(header=dict(values=['Year', 'Total Sales(M$)'],
                                                                            line_color='darkslategray',
                                                                            fill_color='cornflowerblue',
                                                                            align=['left','center'],
                                                                            font=dict(color='white', size=15),
                                                                            height=40),
                                                     cells=dict(values=[[2003,2004,2005], 
                                                                        np.round(list((df.groupby('YEAR_ID').sum()['SALES']/1000000)), 4)],
                                                                        line_color='darkslategray',
                                                                        align=['left', 'center'],
                                                                         fill_color='white',
                                                                        font=dict(color='black', size=15),
                                                
                                                                        height=30))],
                                               layout=dict(template='plotly_dark') ),
                                              style={'width':600, 'height':350,'textAlign':'center'} 
                               )
                     ])
                                         
                                   
                              
                         ])
              ])
        
    
                    

                             
                     
                 
    
@app.callback(Output('graph1', 'figure'),
              [Input('year', 'value')])
def graph1(year):
    figure=px.pie(df[df['YEAR_ID']==year], values='SALES', names='COUNTRY',
          template='plotly_dark',color_discrete_sequence=px.colors.sequential.RdBu, title='Sales For Each Country Per Year')

    return figure
    
@app.callback(Output('graph2', 'figure'),
              [Input('country1', 'value')])
def graph2(country):
    figure=px.bar(df[df['COUNTRY']==country],x='PRODUCTLINE',y='SALES',
                  title='Sales for Each Productline',template='plotly_dark')
    return figure

@app.callback(Output('graph3', 'figure'),
              [Input('country1', 'value')])
def graph3(country):
    
    figure=px.bar(df[df['COUNTRY']==country], x ='MONTH_ID', y='SALES', color = 'PRODUCTLINE', barmode='group',
                  title='Sales for Each Month',template='plotly_dark')

    return figure




In [8]:
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
